In [1]:
pip install tangelo-gc

try:
    import os
except ImportError:
    print ("os error")

SyntaxError: invalid syntax (2325480086.py, line 1)

In [2]:
pip install PySCF

  Using cached pyscf-2.5.0-py3-none-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (3.1 kB)
Using cached pyscf-2.5.0-py3-none-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (47.3 MB)


# Building frozen silicon hexagon

## First, we're building the shape. We find that we're using 24 active molecular orbitals and 24 active electrons.

In [4]:
##########################
from tangelo import SecondQuantizedMolecule as SQMol

# crystal lattice Si
# source: https://www.princeton.edu/~maelabs/mae324/glos324/silicon.htm
# si_xyz = [("Si", (0., 0., 0.)), ("Si", (0., 0., 2.35))]

# put into hexagon
si_xyz = [("Si", (0., 0., 0.)), ("Si", (0., -2.35, 0)), ("Si", (2.35*0.5, 2.35*0.866, 0)),
         ("Si", (2.35*0.5, -2.35*0.866, 0)), ("Si", (-2.35*0.5, 2.35*0.866, 0)), ("Si", (-2.35*0.5, -2.35*0.866, 0))]


si_6311gdp = SQMol(si_xyz, q=0, spin=2)
print(f"{si_6311gdp.n_active_mos} active molecular orbitals")
print(f"{si_6311gdp.n_active_electrons} active electrons")

##########################

basis_sets = [
    "STO-3G",       # Simple zeta, minimal basis.
    "3-21G",        # Double zeta.
    "6-31G",        # Double zeta with more Gaussian primitives.
    "6-31G(d,p)",   # Polarization functions (+ 5 d-orbitals for all atoms except H, +3 p-orbitals for H atoms) added.
    "6-311G(d,p)",  # Triple zeta with polarization functions.
    "6-311+G(d,p)", # Triple zeta with polarization functions and diffuse functions.
    "cc-pvqz",      # Quadruple zeta.
    "cc-pv5z"       # Quintuple zeta.
]

###
si_6311gdp.mo_occ

24 active molecular orbitals
24 active electrons


NPArrayWithTag([2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
                2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
                2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 1.,
                1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

### Let's freeze everything but the valence shell.

In [7]:
from tangelo import SecondQuantizedMolecule as SQMol
si6=  """Si 0.  0. 0.
         Si 0. -2.35 0.
         Si 2.35*0.5 2.35*0.866 0.
         Si 2.35*0.5 -2.35*0.866 0.
         Si -2.35*0.5 2.35*0.866 0.
         Si -2.35*0.5 -2.35*0.866 0. """

# freeze some
fo = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17,18, 19, 20, 21, 22, 23, 24]+[i for i in range(31,50)]

# Runs RHF calculation
mol_si6 = SQMol(si6, q=0, spin=0, basis='6-31g(d,p)', frozen_orbitals=fo, symmetry=True)

print("  #  Energy  Symm Occ")
for i in range(10):
    print(f"{i+1:3d}{mol_si6.mo_energies[i]: 9.4f}  {mol_si6.mo_symm_labels[i]}   {int(mol_si6.mo_occ[i])}")

# Active electrons, Active orbitals
print(f"Number of active electrons: {mol_si6.n_active_electrons}")
print(f"Number of active orbtials: {mol_si6.n_active_mos}")

  #  Energy  Symm Occ
  1 -68.9478  A1   2
  2 -68.8524  A1   2
  3 -68.8116  B2   2
  4 -68.8116  A1   2
  5 -68.8095  A1   2
  6 -68.8094  B2   2
  7  -6.4481  A1   2
  8  -6.2513  B2   2
  9  -6.2317  A1   2
 10  -6.2091  A1   2
Number of active electrons: 12
Number of active orbtials: 64


In [ ]:
# Now, let us try to get the ground energy state.
print("Importing VQE, Ansatze")
from tangelo.algorithms.variational import VQESolver, BuiltInAnsatze
print("Importing FCI")
from tangelo.algorithms.classical import FCISolver

algorithm_resources = dict()

print("Solving FCI")
print(f"CASCI energy = {FCISolver(mol_si6).simulate()}")

# Ground state energy calculation with VQE, reference values with FCI
vqe_options = {"molecule": mol_si6, "ansatz": BuiltInAnsatze.UCCSD}
print("Solving")
vqe_solver = VQESolver(vqe_options)
print("Building")
vqe_solver.build()
vqe_energy = vqe_solver.simulate()
print("\n Ground Singlet state")
print(f"VQE energy = {vqe_energy}")

algorithm_resources["vqe_ground_state"] = vqe_solver.get_resources()

Importing VQE, Ansatze
Importing FCI
Solving FCI
